In [1]:
import numpy as np
import cv2

In [2]:
# Segment the Foreground and Background
def segment_foreground_background(image):
    # Simple thresholding for segmentation
    _, binary_mask = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
    
    # Invert mask to get background
    background = cv2.bitwise_not(binary_mask)

    # Calculate texture features such as entropy
    texture_entropy = -np.sum(background * np.log2(background + 1e-10)) / background.size

    # Calculate variance of background intensity
    background_variance = np.var(background)
    
    return [texture_entropy, background_variance]


In [3]:
# Step 1: Preprocess the Image
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))  # Resize for consistency
    return image

In [4]:
image = preprocess_image("C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Live/Train/16.jpg")
segmented_image = segment_foreground_background(image)
print(segmented_image)

[-1489.6021626739898, 12795.070825517178]


In [6]:
# Extract features
def extract_foreground_background(image_path):
    image = preprocess_image(image_path)
    Background=segment_foreground_background(image)
    return Background

In [7]:
import os 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.svm import SVC

In [8]:
folder_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset'

In [9]:
def load_dataset(folder_path, label):
    feature_list = []
    labels=[]
    
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        features = extract_foreground_background(image_path)
        feature_list.append(features)
        labels.append(label)
    
    return np.array(feature_list), np.array(labels)

In [10]:
live_train_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Live/Train'
live_test_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Live/Test'
spoof_train_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Spoof/Train'
spoof_test_path = 'C:/Users/sadik/OneDrive/Desktop/Liveness detection/dataset/Spoof/Test'

In [11]:
#Extract feature for both live and spoof dataset 
X_live_train,y_live_train = load_dataset(live_train_path, label=1)
X_spoof_train, y_spoof_train = load_dataset(spoof_train_path,label=0)
X_live_test, y_live_test = load_dataset(live_test_path,label=1)
X_spoof_test, y_spoof_test = load_dataset(spoof_test_path,label=0)

In [12]:
# Combine the live and spoof training data
X_train = np.vstack((X_live_train, X_spoof_train))
y_train = np.concatenate((y_live_train, y_spoof_train), axis=0)

# Combine the live and spoof testing data
X_test = np.vstack((X_live_test, X_spoof_test))
y_test = np.concatenate((y_live_test, y_spoof_test), axis=0)

In [13]:
#Scale the features using StandardScaler
ss=StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled=ss.fit_transform(X_test)

In [14]:
# Check shapes to confirm consistency
print("Combined Train Shape:", X_train.shape, y_train.shape)

Combined Train Shape: (480, 2) (480,)


In [15]:
svc = SVC(kernel='rbf', C=1.0, random_state=42)  
svc.fit(X_train_scaled, y_train)

SVC(random_state=42)

In [16]:
import joblib
import sys
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from collections import Counter


class_counts = Counter(y_train)
sys.modules['sklearn.externals.joblib']=joblib
n_splits = min(class_counts.values())
forward_fs_best=sfs(estimator=svc,k_features='best',forward=True,verbose=1,scoring='accuracy',cv=2)


sfs_forward_best=forward_fs_best.fit(X_train_scaled,y_train)

c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
Features: 1/2c:\Users\sadik\anaconda3\envs\liveness\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
Features: 2/2

In [17]:
print('R-Squared value:', sfs_forward_best.k_score_)

R-Squared value: 0.5458333333333334


In [18]:
# Predict on the test data
y_pred = svc.predict(X_test_scaled)

In [19]:
# Determine the unique labels in y_test
unique_labels = np.unique(y_test)

# Generate the confusion matrix with the appropriate labels
conf_matrix = confusion_matrix(y_test, y_pred, labels=unique_labels)

# Handle cases where unique_labels may not contain both 0 and 1
TP = conf_matrix[0][0] if 1 in unique_labels else 0
FN = conf_matrix[0][1] if 1 in unique_labels and len(unique_labels) > 1 else 0
FP = conf_matrix[1][0] if 0 in unique_labels and len(unique_labels) > 1 else 0
TN = conf_matrix[1][1] if 0 in unique_labels else 0

# Print the results
print("True Positive:", TP)
print("False Negative:", FN)
print("False Positive:", FP)
print("True Negative:", TN)
print("Accuracy of the SVM:", accuracy_score(y_test, y_pred))
print("Precision of the SVM:", precision_score(y_test, y_pred, pos_label=1) if 1 in unique_labels else 'N/A')
print("Recall of the SVM:", recall_score(y_test, y_pred, pos_label=1) if 1 in unique_labels else 'N/A')


True Positive: 41
False Negative: 19
False Positive: 14
True Negative: 46
Accuracy of the SVM: 0.725
Precision of the SVM: 0.7076923076923077
Recall of the SVM: 0.7666666666666667
